# MACS-30123 Final Project: Data Collection
Baotong Zhang
baotongzh@uchicago.edu

In [1]:
import pandas as pd
import csv
from tqdm import tqdm
import re

In [2]:
songs = pd.read_csv('billboard.csv')

In [3]:
songs.shape

(330087, 7)

In [4]:
songs.head(10)

,date,rank,song,artist,last-week,peak-rank,weeks-on-board
0,2021-11-06,1,Easy On Me,Adele,1.0,1,3
1,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16
2,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14
3,2021-11-06,4,Fancy Like,Walker Hayes,4.0,3,19
4,2021-11-06,5,Bad Habits,Ed Sheeran,5.0,2,18
5,2021-11-06,6,Way 2 Sexy,Drake Featuring Future & Young Thug,6.0,1,8
6,2021-11-06,7,Shivers,Ed Sheeran,9.0,7,7
7,2021-11-06,8,Good 4 U,Olivia Rodrigo,7.0,1,24
8,2021-11-06,9,Need To Know,Doja Cat,11.0,9,20
9,2021-11-06,10,Levitating,Dua Lipa,8.0,2,56


In [5]:
songs['year'] = pd.DatetimeIndex(songs['date']).year

In [6]:
songs.drop(['date', 'rank', 'last-week', 'peak-rank', 'weeks-on-board'], axis=1, inplace=True)
songs.drop_duplicates(inplace=True)

In [7]:
songs.shape

(36083, 3)

In [8]:
sampled_df = songs.groupby('year').sample(n=150, random_state=42, replace=False)

In [9]:
sampled_df.groupby('year').count()

,song,artist
year,,
1958,150,150
1959,150,150
1960,150,150
1961,150,150
1962,150,150
...,...,...
2017,150,150
2018,150,150
2019,150,150


In [10]:
sampled_df.shape

(9600, 3)

In [11]:
sampled_df.head()

,song,artist,year
329256,Cerveza,Boots Brown And His Blockbusters,1958
328165,Tunnel Of Love,Doris Day,1958
327942,Lucky Ladybug,Billy & Lillie,1958
328475,It Don't Hurt No More,Nappy Brown,1958
328084,The Reason,The 5 Chanels,1958


In [12]:
year_df = sampled_df[sampled_df['year'] == 1958]

In [13]:
year_df.head()

,song,artist,year
329256,Cerveza,Boots Brown And His Blockbusters,1958
328165,Tunnel Of Love,Doris Day,1958
327942,Lucky Ladybug,Billy & Lillie,1958
328475,It Don't Hurt No More,Nappy Brown,1958
328084,The Reason,The 5 Chanels,1958


In [15]:
token = 'JyD9-_LdqpyHilUA3esIXZt0Dp4Ftcc4TMo9cq7b4q2zLVfDDR1AYtIoXsXnf1DS'

In [16]:
genius = Genius(token)
genius.verbose = False # Turn off status messages
genius.remove_section_headers = True # Remove section headers (e.g. [Chorus]) from lyrics when searching
genius.skip_non_songs = False # Include hits thought to be non-songs (e.g. track lists)
genius.excluded_terms = ["(Remix)", "(Live)"] # Exclude songs with these words in their title

In [17]:
def scraper():
    with open('song_info.csv', 'w', newline="", encoding='utf-8') as csvfile:
        output = csv.writer(csvfile)
        output.writerow(['Year', 'Artist', 'Song Name', 'Lyrics'])
        for year in range(1958, 2022):
            year_df = sampled_df[sampled_df['year'] == year]
            for i in tqdm(range(150)):
                song_name = year_df.iloc[i, 0]
                song_artist = re.split('& | Featuring | And', year_df.iloc[i, 1]) + [year_df.iloc[i, 1]]
                try:
                    for j in song_artist:
                        if genius.search_artist(j , max_songs=1, sort="title"):
                            artist = genius.search_artist(j , max_songs=1, sort="title")
                            try:
                                song = artist.song(song_name)
                                lyrics = song.lyrics
                                output.writerow([year, year_df.iloc[i, 1], song_name, lyrics])
                                break
                            except:
                                continue
                        else:
                            continue
                except:
                    pass

In [18]:
scraper()

100%|██████████| 150/150 [21:27<00:00,  8.58s/it]


In [19]:
lyrics_df = pd.read_csv('song_info.csv')

In [23]:
lyrics_df.head()

,Year,Artist,Song Name,Lyrics
0,1958,Boots Brown And His Blockbusters,Cerveza,Cerveza LyricsInstrumental\nInstrumentalEmbed
1,1958,Doris Day,Tunnel Of Love,Tunnel Of Love LyricsGetting crazy on the walt...
2,1958,Billy & Lillie,Lucky Ladybug,"Lucky Ladybug LyricsLucky Ladybug, lucky ladyb..."
3,1958,Nappy Brown,It Don't Hurt No More,Damn Lyrics\nOhhhhhhhhh! That's right\nYour gi...
4,1958,Cozy Cole,Topsy I,NAGASAKI LyricsHook:\nIm back in this bitch an...


In [26]:
lyrics_df.groupby('Year').count()

,Artist,Song Name,Lyrics
Year,,,
1958,122,122,120
1959,115,115,111
1960,129,129,127
1961,125,125,124
1962,124,124,121
...,...,...,...
2017,148,148,148
2018,147,147,147
2019,145,145,145
